<img width="8%" alt="Growth" src="https://naasai-public.s3.eu-west-3.amazonaws.com/abi-demo/growth_marketing.png" style="border-radius: 15%">

# Growth - Create ORGANIZATIONS database

**Tags:** #growth #googlesheets #gsheet #data #naas_drivers #companies #organizations #openai #linkedin

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook updates your people organizations database enrich them with data from LinkedIn.

## Input

### Import libraries

In [ ]:
from naas_drivers import gsheet, linkedin
import pandas as pd
import os
from datetime import date, datetime
import naas_data_product
import openai
import time
from googlesearch import search
import re

### Setup variables
**Inputs**
- `entity_index`: Entity index.
- `entity_dir`: Entity directory.
- `input_dir`: Input directory to retrieve file from.
- `file_interactions`: Name of the file to be retrieved.
- `file_people`: Name of the file to be retrieved.
- `li_at`: Cookie used to authenticate Members and API clients.
- `JSESSIONID`: Cookie used for Cross Site Request Forgery (CSRF) protection and URL signature validation.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_people_organizations`: Google Sheets sheet name storing organizations.
- `sheet_people`: Google Sheets sheet name storing people.

**Outputs**
- `output_dir`: Output directory to save file to.
- `sheet_people_organizations`: Output file name to save as picke.
- `datalake_dir`: Datalake directory (outputs folder from abi project).

In [ ]:
# Inputs
entity_index = "0"
entity_dir = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_index), "entity_dir")
input_dir = os.path.join(entity_dir, "growth-engine", date.today().isoformat())
file_interactions = "interactions"
file_people = "people"
li_at = os.environ.get("LINKEDIN_LI_AT") or naas.secret.get("LINKEDIN_LI_AT")
JSESSIONID = os.environ.get("LINKEDIN_JSESSIONID") or naas.secret.get("LINKEDIN_JSESSIONID")
spreadsheet_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", entity_index), "abi_spreadsheet")
sheet_people_organizations = "ORGANIZATIONS"
sheet_people = "PEOPLE"

# Outputs
output_dir = os.path.join(entity_dir, "growth-engine", date.today().isoformat())
file_organizations = "organizations"
datalake_dir = naas_data_product.OUTPUTS_PATH

## Model

### Get organizations

In [ ]:
df_init = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_people_organizations)
if not isinstance(df_init, pd.DataFrame):
    df_init = pd.DataFrame()
print("- Organizations (init):", len(df_init))
# df_init.head(3)

### Get interactions

In [ ]:
df_interactions = pload(input_dir, file_interactions)    
print('- Interactions:', len(df_interactions))
# df_interactions.head(3)

### Get people

In [ ]:
df_people = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_people)
if not isinstance(df_people, pd.DataFrame):
    df_people = pd.DataFrame()
print("- People:", len(df_people))
# df_people.head(3)

### Extract organizations from interactions

In [ ]:
def get_linkedin_url(keyword, urls):
    # Init linkedinbio
    url = "NA"
    urls[keyword] = url
    
    # Create query
    query = f"{keyword.replace(' ', '+')}+LinkedIn+company"
    print("Google query: ", query)
    # Search in Google
    for i in search(query, tld="com", num=10, stop=10, pause=2):
        pattern = f"https:\/\/.+.linkedin.com\/company\/.([^?])+"
        result = re.search(pattern, i)
        # Avoid Too Many Requests Error
        time.sleep(5)
        # Return value if result is not None
        if result != None:
            url = result.group(0).replace(" ", "")
            urls[keyword] = url
            break
            
    pdump(output_dir, urls, "org_lk_urls")
    return url

def create_db_organizations(
    df_init,
    df_interactions,
    df_people,
    output_dir
):
    # -> Direct interactions
    df_org = df_interactions[~df_interactions["PROFILE_URL"].str.contains("https://www.linkedin.com/in/.+")]
    df_score = df_org.copy()
    df_direct = df_org.copy()
    
    # Get interactions score by profile
    df_score = df_score.groupby(["PROFILE_URL"], as_index=False).agg({"SCORE": "sum"})
    
    # Get profile information with last content interaction
    to_keep = [
        "PROFILE_URL",
        "FULLNAME",
    ]
    df_direct = df_direct[to_keep].drop_duplicates().drop_duplicates(["PROFILE_URL"])
    
    # Merge dfs
    df_d = pd.merge(df_score, df_direct, how="left")
    
    # Cleaning: Remove emojis from name
    df_d["FULLNAME"] = df_d.apply(lambda row: remove_emojis(row["FULLNAME"]), axis=1)
    
    # Cleaning: Rename columns
    to_rename = {
        "FULLNAME": "ORGANIZATION",
        "PROFILE_URL": "LINKEDIN_URL",
        "SCORE": "DIRECT_INTERACTIONS",
    }
    df_d = df_d.rename(columns=to_rename)
    
    # -> Indirect interactions
    df_i = df_people.copy()
    to_group = [
        "ORGANIZATION",
    ]
    to_agg = {
        "INTERACTION_SCORE": "sum"
    }
    to_rename = {
        "INTERACTION_SCORE": "INDIRECT_INTERACTIONS"
    }
    df_i = df_i.groupby(to_group, as_index=False).agg(to_agg).rename(columns=to_rename)
    
    # -> Concat company list dfs
    df = pd.concat([df_d[["ORGANIZATION"]], df_i[["ORGANIZATION"]]]).drop_duplicates("ORGANIZATION")
    df["ORGANIZATION"] = df["ORGANIZATION"].astype(str)
    df = df.sort_values(by="ORGANIZATION").reset_index(drop=True)

    # -> Enrich with scores
    fillna = {
        "LINKEDIN_URL": "TBD",
        "DIRECT_INTERACTIONS": 0,
        "INDIRECT_INTERACTIONS": 0
    }
    df = pd.merge(df, df_d, how="left").merge(df_i, how="left").fillna(fillna)
    df["INTERACTION_SCORE"] = df["DIRECT_INTERACTIONS"] * 5 + df["INDIRECT_INTERACTIONS"]
    df = df.sort_values(by=["INTERACTION_SCORE"], ascending=[False])
    df = df[~df["ORGANIZATION"].isin(["NA", "TBD", "None", "n/a", 'Not Found', 'UNKNOWN', 'ERROR_LINKEDIN_ENRICHMENT'])]
        
    # Get meta data from existing people
    col_ref = [
        "ORGANIZATION",
        "LINKEDIN_URL",
        "INDUSTRY",
        "CITY",
        "COUNTRY",
        "STAFF_RANGE",
        "STAFF_RANGE_NAME",
        "STAFF_COUNT",
        "FOLLOWERS_COUNT",
        "WEBSITE",
        "TAGLINE",
        "DESCRIPTION",
        "ORG_NAME",
        "ORG_LINKEDIN_ID",
        "ORG_LINKEDIN_URL",
        "CRM_ORG_ID"
    ]
    for c in col_ref:
        # If columns does not exist, init value to be determined (TBD)
        if not c in df_init.columns:
            df_init[c] = "TBD"
    ref = df_init[col_ref].drop_duplicates("ORGANIZATION")
    
    # Merge to get meta data
    df = pd.merge(df, ref, how="left", on="ORGANIZATION").fillna("TBD")
    df.loc[df["LINKEDIN_URL_x"] != "TBD", "LINKEDIN_URL"] = df["LINKEDIN_URL_x"]
    df.loc[df["LINKEDIN_URL_x"] == "TBD", "LINKEDIN_URL"] = df["LINKEDIN_URL_y"]
    
    # Cleaning
    to_order = [
        'ORGANIZATION',
        'INDUSTRY',
        'CITY',
        'COUNTRY',
        'STAFF_RANGE_NAME',
        'INTERACTION_SCORE',
        'DIRECT_INTERACTIONS',
        'INDIRECT_INTERACTIONS',
        'STAFF_RANGE',
        'STAFF_COUNT',
        'FOLLOWERS_COUNT',
        'WEBSITE',
        'TAGLINE',
        'DESCRIPTION',
        'ORG_NAME',
        'ORG_LINKEDIN_ID',
        'ORG_LINKEDIN_URL',
        'LINKEDIN_URL',
        "CRM_ORG_ID"
    ]
    df = df[to_order]
    
    # -> Enrich with LinkedIn URL from Google Search
    org_lk_urls = get_dict_from_df(df, "LINKEDIN_URL", "ORGANIZATION", "org_lk_urls", output_dir)

    # Loop on LinkedIn TBD
    filter_df = df[(df["LINKEDIN_URL"] == "TBD") & ~(df["ORGANIZATION"].isin(org_lk_urls.keys()))]
    print("-> New organizations:", len(filter_df))
    
    count = 1
    if len(filter_df) > 0:
        for row in filter_df.itertuples():
            index = row.Index
            organization = row.ORGANIZATION
            linkedin_url = row.LINKEDIN_URL

            # Update LINKEDIN_URL column
            print()
            print(f"{count} - 🤖 Google Search - Finding LinkedIn URL for '{organization}'")
            linkedin_url = get_linkedin_url(organization, org_lk_urls)
            print("- LinkedIn URL:", linkedin_url)
            count += 1
            
    # Map Organizations LinkedIn URL    
    df["LINKEDIN_URL"] = df["ORGANIZATION"].map(org_lk_urls)
    
    # Save database
    pdump(output_dir, df, "organizations_init")
    return df.reset_index(drop=True)

db_organizations = create_db_organizations(
    df_init,
    df_interactions,
    df_people,
    output_dir,
)
print("- Organizations:", len(db_organizations))
db_organizations.head(1)

### Enrich organizations with LinkedIn data

In [ ]:
def enrich_organizations(
    df_init,
    output_dir,
    limit_linkedin=30
):
    # Init
    df = df_init.copy()
    
    # Filter data
    filter_df = df[
        ~(df["LINKEDIN_URL"].isin(["NA"])) &
        (df["ORG_LINKEDIN_ID"].isin(["TBD"]))
    ]
    print("-> Organization to be updated:", len(filter_df))
    
    # Loop on companies
    count = 1
    call_linkedin = 0
    for row in filter_df.itertuples():
        index = row.Index
        organization = row.ORGANIZATION
        linkedin_url = str(row.LINKEDIN_URL)
        interaction_score = row.INTERACTION_SCORE
        organization_id = row.ORG_LINKEDIN_ID
        
        if "company" in linkedin_url and organization_id == "TBD" and call_linkedin < limit_linkedin and (interaction_score >= 3 or call_linkedin < limit_linkedin):
            print()
            print(f"{count} - 🕸️ LinkedIn - Enrich data for '{organization}': {int(interaction_score)} ({linkedin_url})")
            linkedin_dir = os.path.join(datalake_dir, "datalake", "linkedin", "organizations")            
            linkedin_id = linkedin_url.split("/company/")[1].split("/")[0]
            tmp_df = pload(linkedin_dir, f"{linkedin_id}_linkedin_company_info")
            if tmp_df is None:
                try:
                    tmp_df = linkedin.connect(li_at, JSESSIONID).company.get_info(linkedin_url)
                    pdump(linkedin_dir, tmp_df, f"{linkedin_id}_linkedin_company_info")
                    time.sleep(2)
                    call_linkedin += 1
                    print("- ⚠️ LinkedIn call:", call_linkedin)
                except Exception as e:
                    print(e)
                    if e.response.status_code == 302:
                        call_linkedin = limit_linkedin
                    tmp_df = pd.DataFrame()
                
            if len(tmp_df) > 0:                
                df.loc[index, "ORG_LINKEDIN_ID"] = tmp_df.loc[0, "COMPANY_ID"]
                df.loc[index, "ORG_NAME"] = tmp_df.loc[0, "COMPANY_NAME"]
                df.loc[index, "ORG_LINKEDIN_URL"] = tmp_df.loc[0, "COMPANY_URL"]
                df.loc[index, "INDUSTRY"] = tmp_df.loc[0, "INDUSTRY"]
                df.loc[index, "STAFF_COUNT"] = tmp_df.loc[0, "STAFF_COUNT"]
                df.loc[index, "STAFF_RANGE"] = tmp_df.loc[0, "STAFF_RANGE"]
                df.loc[index, "FOLLOWERS_COUNT"] = tmp_df.loc[0, "FOLLOWER_COUNT"]
                df.loc[index, "COUNTRY"] = get_country_name(tmp_df.loc[0, "COUNTRY"])
                df.loc[index, "CITY"] = tmp_df.loc[0, "CITY"]
                df.loc[index, "WEBSITE"] = tmp_df.loc[0, "WEBSITE"]
                df.loc[index, "TAGLINE"] = tmp_df.loc[0, "TAGLINE"]
                df.loc[index, "DESCRIPTION"] = tmp_df.loc[0, "DESCRIPTION"]
            else:
                df.loc[index, "ORG_LINKEDIN_ID"] = "Not Found"
                df.loc[index, "ORG_NAME"] = "Not Found"
                df.loc[index, "ORG_LINKEDIN_URL"] = "Not Found"
                df.loc[index, "INDUSTRY"] = "Not Found"
                df.loc[index, "STAFF_COUNT"] = "Not Found"
                df.loc[index, "STAFF_RANGE"] = "Not Found"
                df.loc[index, "FOLLOWERS_COUNT"] = "Not Found"
                df.loc[index, "COUNTRY"] = "Not Found"
                df.loc[index, "CITY"] = "Not Found"
                df.loc[index, "WEBSITE"] = "Not Found"
                df.loc[index, "TAGLINE"] = "Not Found"
                df.loc[index, "DESCRIPTION"] = "Not Found"
            
            if call_linkedin >= limit_linkedin:
                print("🛑 Call LinkedIn reached:", limit_linkedin)
            count += 1
    # Cleaning
    df.STAFF_RANGE = df.STAFF_RANGE.str.replace("-None", ">")
    df = df.replace("nan", "NA").replace("na", "NA").replace("None", "NA")
    if "STAFF_RANGE" in df:
        staff_maping = {
            "NA": "NA",
            "UNKNOWN": "NA",
            "TBD": "TBD",
            "0-1": "Solopreneur (0-1)",
            "2-10": "Micro Team (2-10)",
            "11-50": "Small Company (11-50)",
            "51-200": "Medium Company (51-200)",
            "201-500": "Large Company (201-500)",
            "501-1000":	"Enterprise Level (501-1000)",
            "1001-5000": "Major Corporation (1001-5000)",
            "5001-10000": "Global Corporation (5001-10000)",
            "10001>": "Mega Corporation (10001>)",
        }
        df["STAFF_RANGE_NAME"] = df["STAFF_RANGE"].map(staff_maping)
        
    astypes = {
        "INTERACTION_SCORE": int,
        "DIRECT_INTERACTIONS": int,
        "INDIRECT_INTERACTIONS": int,        
    }
    df = df.astype(astypes)
    df = df.sort_values(by=["INTERACTION_SCORE", "ORGANIZATION"], ascending=[False, True])
    return df.reset_index(drop=True)

df_organizations = enrich_organizations(
    db_organizations,
    output_dir,
)
df_organizations.head(1)

### Update Organization names to remove duplicates

In [ ]:
def update_dfs_from_dict(
    org_dict,
    df_p,
    df_org
):
    if len(org_dict) > 0:
        # Update names in Organizations db 
        df_org["ORGANIZATION"] = df_org.apply(lambda row: org_dict.get(row["ORGANIZATION"]) if row["ORGANIZATION"] in org_dict else row["ORGANIZATION"], axis=1)
        
        # Update names in People db
        df_p["ORGANIZATION"] = df_p.apply(lambda row: org_dict.get(row["ORGANIZATION"]) if row["ORGANIZATION"] in org_dict else row["ORGANIZATION"], axis=1)
    return df_org, df_p
    
def update_org_names(
    df_organizations,
    df_people,
    df_interactions, 
    output_dir
):    
    # Init
    df_org = df_organizations.copy()
    df_p = df_people.copy()
    
    # Update org name with company name from LinkedIn
    df = df_organizations.copy()
    df = df[(df["ORGANIZATION"] != df["ORG_NAME"]) & (df["ORG_LINKEDIN_ID"] != "TBD")]
    org_names_1 = get_dict_from_df(df, "ORG_NAME", "ORGANIZATION", f"organizations_names", output_dir)
    print("-> New Organization to be updated in People db:", len(org_names_1))
    if len(org_names_1) > 0:
        df_org, df_p = update_dfs_from_dict(org_names_1, df_p, df_org)
        org_lk_urls = pload(output_dir, "org_lk_urls")
        for key, value in org_names_1.items():
            key = key.replace("Not Found", "")
            org_lk_urls[value] = org_lk_urls[key]
        pdump(output_dir, org_lk_urls, "org_lk_urls")
    
    # Update org with data tbd
    org_names_2 = {}
    
    # Get org name with data tbd with same LinkedIn URL as org found
    df1 = df_org.copy()
    df1 = df1[(df1["LINKEDIN_URL"] != "NA") & (df1["ORG_LINKEDIN_ID"] != "TBD")]
    organizations_org_urls = get_dict_from_df(df1, "ORGANIZATION", "LINKEDIN_URL", f"organizations_org_urls", output_dir)

    # Get org name with data tbd with similar LinkedIn URL
    df2 = df_org.copy()
    df2 = df2[(df2["LINKEDIN_URL"] != "NA") & (df2["ORG_LINKEDIN_ID"] == "TBD")]
    counts = df2["LINKEDIN_URL"].value_counts()
    filtered_counts = counts.loc[counts > 1].to_dict()
    df3 = df_org.copy()
    df3 = df3[(df3["LINKEDIN_URL"].isin(filtered_counts))].sort_values(by=["LINKEDIN_URL", "ORGANIZATION"]).drop_duplicates("LINKEDIN_URL")

    # Get org with data to be enriched
    df_tbd = df_org.copy()
    df_tbd = df_tbd[(df_tbd["ORG_LINKEDIN_ID"] == "TBD")]
    for row in df_tbd.itertuples():
        org = row.ORGANIZATION
        url = row.LINKEDIN_URL
        if row.LINKEDIN_URL in organizations_org_urls:
            org_name = df1.loc[df1["LINKEDIN_URL"] == url, "ORG_NAME"].values[0]
            org_names_2[org] = org_name
        if row.LINKEDIN_URL in filtered_counts and not org in org_names_2:
            org_name = df3.loc[df3["LINKEDIN_URL"] == url, "ORGANIZATION"].values[0]
            org_names_2[org] = org_name
            
    print("-> Organization duplicated:", len(org_names_2))
    if len(org_names_2) > 0:
        df_org, df_p = update_dfs_from_dict(org_names_2, df_p, df_org)
        org_lk_urls = pload(output_dir, "org_lk_urls")
        for key, value in org_names_2.items():
            org_lk_urls[value] = org_lk_urls[key]
        pdump(output_dir, org_lk_urls, "org_lk_urls")
    
    # Aggregate new orgs
    if len(org_names_1) + len(org_names_2) > 0:
        df_org = enrich_organizations(
                create_db_organizations(
                df_org,
                df_interactions,
                df_p,
                output_dir,
            ),
            output_dir
        )
    return df_org, df_p

df_organizations_u, df_people_u = update_org_names(df_organizations, df_people, df_interactions, output_dir)
print("- Organizations (updated):", len(df_organizations_u))
print("- People (updated):", len(df_people_u))

## Output

### Save data "Companies"

In [ ]:
pdump(output_dir, df_organizations_u, file_organizations)

### Send "Companies" to spreadsheet

In [ ]:
send_data_to_gsheet(df_organizations_u, df_init, spreadsheet_url, sheet_people_organizations)

### Update "People" spreadsheet

In [ ]:
send_data_to_gsheet(df_people_u, df_people, spreadsheet_url, sheet_people)